In [ ]:
# import libraries
import contextlib
from datetime import timedelta
from datetime import datetime
import os
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
import mlflow
import pandas as pd
import xgboost as xgb
from helpers import convert_numerical_columns_to_float, read_csv_from_minio, transition_best_model_version_to_prod
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from hyperopt.pyll import scope
from sklearn.metrics import (accuracy_score, f1_score, precision_score,
                             recall_score)
from sklearn.model_selection import train_test_split
from minio import Minio


from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from datetime import datetime, timedelta
from minio import Minio
import pandas as pd
import pickle

#def initialize_minio_client(**kwargs):
    #minio_client = Minio(
    #    endpoint='20.224.70.229:9001',
    #    access_key='abdessamadbaahmed',
    #    secret_key='baahmedabdessamad',
    #   secure=False
    #)
    # Serialize the Minio client object using pickle
    #minio_client_pickled = pickle.dumps(minio_client)
    # Store the serialized Minio client object as an Airflow variable
    #kwargs['ti'].xcom_push(key='minio_client', value=minio_client_pickled)
    

def read_csv_from_minio(**kwargs):
    minio_client = Minio(
        endpoint='20.224.70.229:9001',
        access_key='abdessamadbaahmed',
        secret_key='baahmedabdessamad',
        secure=False
    )
    try:
        # Get the object data
        data = minio_client.get_object(kwargs['bucket_name'], kwargs['object_name'])
        df = pd.read_csv(data)
        kwargs['ti'].xcom_push(key='dataframe', value=df)
    except Exception as e:
        print(e)

        
def preprocessing_data(**kwargs):
    df = kwargs['ti'].xcom_pull(key='dataframe', task_ids='read_csv_from_minio')
    with contextlib.suppress(Exception):
        # dropping the name column
        df.drop(["Name"], axis=1, inplace=True)
        
    # replacing the missing values with the mean of the column "3P%"
    df["3P%"].fillna(df["3P%"].mean(), inplace=True)

    # define features and target
    X_train = df.drop("TARGET_5Yrs", axis=1)
    y_train = df["TARGET_5Yrs"]

    # splitting the data into train and validation
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
    )

    # X_train = xgb.DMatrix(X_train, label=y_train)
    # X_val = xgb.DMatrix(X_val, label=y_val)

    return X_train, X_val, y_train, y_val


default_args = {
    'owner': 'me',
    'start_date': datetime(2022, 1, 1),
    'depends_on_past': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

with DAG('testing_dag', default_args=default_args, schedule_interval=timedelta(hours=1)) as dag:

    read_csv_from_minio_task = PythonOperator(
        task_id='read_csv_from_minio',
        python_callable=read_csv_from_minio,
        op_kwargs={'bucket_name': 'nba-investment-data', 'object_name': 'nba_logreg_preprocessed.csv'},
        provide_context=True,
        xcom_push=True,
    )
    
    preprocessing_data_task = PythonOperator(
        task_id='preprocessing_data',
        python_callable=preprocessing_data,
        provide_context=True,
    )
    
    read_csv_from_minio_task >> preprocessing_data_task

    
    
    
    
    
    

read_csv_from_minio_task


In [2]:
from minio import Minio
import pandas as pd 

minio_client = Minio(
endpoint='20.224.70.229:9000',
access_key='abdessamadbaahmed',
secret_key='baahmedabdessamad',
secure=False
)

try:
    # Get the object data
    data = minio_client.get_object('nba-investment-data', 'nba_logreg_preprocessed.csv')
    df = pd.read_csv(data)

except Exception as e:
    print(e)
    

In [3]:
df

,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs
0,36,27.4,7.4,2.6,7.6,34.7,0.5,2.1,25.0,1.6,2.3,69.9,0.7,3.4,4.1,1.9,0.4,0.4,1.3,0.0
1,35,26.9,7.2,2.0,6.7,29.6,0.7,2.8,23.5,2.6,3.4,76.5,0.5,2.0,2.4,3.7,1.1,0.5,1.6,0.0
2,74,15.3,5.2,2.0,4.7,42.2,0.4,1.7,24.4,0.9,1.3,67.0,0.5,1.7,2.2,1.0,0.5,0.3,1.0,0.0
3,58,11.6,5.7,2.3,5.5,42.6,0.1,0.5,22.6,0.9,1.3,68.9,1.0,0.9,1.9,0.8,0.6,0.1,1.0,1.0
4,48,11.5,4.5,1.6,3.0,52.4,0.0,0.1,0.0,1.3,1.9,67.4,1.0,1.5,2.5,0.3,0.3,0.4,0.8,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335,80,15.8,4.3,1.6,3.6,43.3,0.0,0.2,14.3,1.2,1.5,79.2,0.4,0.8,1.2,2.5,0.6,0.2,0.8,0.0
1336,68,12.6,3.9,1.5,4.1,35.8,0.1,0.7,16.7,0.8,1.0,79.4,0.4,1.1,1.5,2.3,0.8,0.0,1.3,1.0
1337,43,12.1,5.4,2.2,3.9,55.0,0.0,0.0,0.0,1.0,1.6,64.3,1.5,2.3,3.8,0.3,0.3,0.4,0.9,0.0
1338,52,12.0,4.5,1.7,3.8,43.9,0.0,0.2,10.0,1.2,1.8,62.5,0.2,0.4,0.7,2.2,0.4,0.1,0.8,1.0


In [ ]:
# import libraries
import contextlib
from datetime import timedelta
from datetime import datetime
import os
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
import mlflow
import pandas as pd
import xgboost as xgb
from helpers import convert_numerical_columns_to_float, read_csv_from_minio, transition_best_model_version_to_prod
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from hyperopt.pyll import scope
from sklearn.metrics import (accuracy_score, f1_score, precision_score,
                             recall_score)
from sklearn.model_selection import train_test_split
from minio import Minio


from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from datetime import datetime, timedelta
from minio import Minio
import pandas as pd
import pickle

#def initialize_minio_client(**kwargs):
    #minio_client = Minio(
    #    endpoint='20.224.70.229:9001',
    #    access_key='abdessamadbaahmed',
    #    secret_key='baahmedabdessamad',
    #   secure=False
    #)
    # Serialize the Minio client object using pickle
    #minio_client_pickled = pickle.dumps(minio_client)
    # Store the serialized Minio client object as an Airflow variable
    #kwargs['ti'].xcom_push(key='minio_client', value=minio_client_pickled)
    
        
        
def read_data_preprocess_data(**kwargs):
    
    minio_client = Minio(
    endpoint='20.224.70.229:9000',
    access_key='abdessamadbaahmed',
    secret_key='baahmedabdessamad',
    secure=False
    )
    
    try:
        # Get the object data
        data = minio_client.get_object(kwargs['bucket_name'], kwargs['object_name'])
        df = pd.read_csv(data)
        print(df)
    except Exception as e:
        print(e)
    
    with contextlib.suppress(Exception):
        # dropping the name column
        df.drop(["Name"], axis=1, inplace=True)
        
    # replacing the missing values with the mean of the column "3P%"
    df["3P%"].fillna(df["3P%"].mean(), inplace=True)
    

    # upload the preprocessed data to minio server
    minio_client.fput_object(bucket_name='nba-investment-data', object_name='nba_logreg_preprocessed.csv', file_path='nba_logreg_processed.csv')
    
    # splitting the data into train and test
    train_set, test_set = train_test_split(df, test_size=0.2, random_state=42, stratify=df["TARGET_5Yrs"])
    
    # save the train and test sets
    train_set.to_csv("nba_logreg_processed_train.csv", index=False)
    test_set.to_csv("nba_logreg_processed_test.csv", index=False)
    
    #upload the train and test sets to minio server
    minio_client.fput_object(bucket_name='nba-investment-data', object_name='nba_logreg_train.csv', file_path='nba_logreg_processed_train.csv')
    minio_client.fput_object(bucket_name='nba-investment-data', object_name='nba_logreg_test.csv', file_path='nba_logreg_processed_test.csv')
    
        
def search_best_hyper_params(**kwargs):
    """
    this function trains the model using hyperparameter tuning

    :param train: training data
    :param valid: validation data
    :param y_val: target column of the validation data

    :return: best hyperparameters
    """
    
    minio_client = Minio(
    endpoint='20.224.70.229:9000',
    access_key='abdessamadbaahmed',
    secret_key='baahmedabdessamad',
    secure=False
    )
    
    train = minio_client.get_object(kwargs['bucket_name'], kwargs['object_name'])

    # define features and target
    X_train = train.drop("TARGET_5Yrs", axis=1)
    y_train = train["TARGET_5Yrs"]

    # splitting the data into train and validation
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
    )
    
    # create the training and validation data for the model in the xgboost format
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)
    
    # Define the objective function for the hyperparameter optimization
    def objective(params, metric=kwargs["metric"]):
        with mlflow.start_run():
            # Set the model and the search space in the run metadata
            mlflow.set_tag("model", "xgboost")
            mlflow.set_tag("holdout_set", "validation set")
            mlflow.log_params(params)

            # Train the XGBoost model using the specified hyperparameters
            booster = xgb.train(
                params=params,  # Hyperparameters
                dtrain=train,  # Training data
                num_boost_round=1000,  # Train for 1000 rounds
                evals=[
                    (valid, "validation")
                ],  # Evaluate on the validation data at each iteration of training
                early_stopping_rounds=50,  # Stop training if the validation score does not improve for 50 rounds
            )

            # Make predictions on the validation data
            y_pred = booster.predict(valid).round()

            # Calculate the evaluation scores
            accuracy = accuracy_score(y_val, y_pred)
            precision = precision_score(y_val, y_pred)
            recall = recall_score(y_val, y_pred)
            f1 = f1_score(y_val, y_pred)

            if metric in ["f1_score", "f1"]:
                metric = f1
            elif metric == "accuracy":
                metric = accuracy
            elif metric == "precision":
                metric = precision
            elif metric == "recall":
                metric = recall

            # Log the evaluation scores to MLFlow
            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("f1_score", f1)

        return {"loss": 1 - metric, "status": STATUS_OK}  # Minimize the negative F1 score

    # Define the search space for the hyperparameters
    search_space = {
        "max_depth": scope.int(hp.quniform("max_depth", 4, 200, 1)),
        "learning_rate": hp.loguniform("learning_rate", -3, 0),
        "reg_alpha": hp.loguniform("reg_alpha", -5, -1),
        "reg_lambda": hp.loguniform("reg_lambda", -6, -1),
        "min_child_weight": hp.loguniform("min_child_weight", -1, 3),
        "objective": "binary:logistic",
        "seed": 42,
    }

    # Perform the hyperparameter optimization using the Tree Parzen Estimator algorithm
    best_result = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=50,
        trials=Trials(),
    )

    best_result["max_depth"] = int(best_result["max_depth"])
    best_result["objective"] = "binary:logistic"
    best_result["seed"] = 42

    print(best_result)




default_args = {
    'owner': 'me',
    'start_date': datetime(2022, 1, 1),
    'depends_on_past': False,
    'retries': 1,
    'retry_delay': timedelta(seconds=20),
    'catchup': False,
    'email': ['email@example.com'],

}


with DAG('testing_dag_4', default_args=default_args, schedule_interval=timedelta(hours=20)) as dag:

    read_data_preprocess_data_task = PythonOperator(
        task_id='read_data_preprocess_data_task',
        python_callable=read_data_preprocess_data,
        provide_context=True,
        op_kwargs={'bucket_name': 'nba-investment-data', 'object_name': 'nba_logreg_raw.csv'}
       # xcom_push=True,
    )
    
    search_best_hyper_params_task = PythonOperator(
        task_id='search_best_hyper_params_task',
        python_callable=search_best_hyper_params,
        provide_context=True,
        op_kwargs={'bucket_name': 'nba-investment-data', 'object_name': 'nba_logreg_processed_train.csv', 'metric': 'f1_score'}
    
    )
    
    read_data_preprocess_data >> search_best_hyper_params_task
